In [1]:
!pip install pgl easydict

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 7.9MB 13.8MB/s eta 0:00:01
     |████████████████████████████████| 51kB 22.8MB/s eta 0:00:01
     |████████████████████████████████| 81kB 20.3MB/s eta 0:00:01
  Created wheel for easydict: filename=easydict-1.9-cp37-none-any.whl size=6350 sha256=9cd44225ba5c8576e092155163212584d79b2b3b0168592deff31db2af077d36
  Stored in directory: /home/aistudio/.cache/pip/wheels/35/8b/38/7327c27cd3d4590ffa75b98030bd3828e68b8bb3d599573163
Successfully built easydict


In [2]:
from collections import namedtuple
import pgl
import paddle.fluid as fluid
import numpy as np
import time
import pandas as pd
import random
from easydict import EasyDict as edict

config = {
    "model_name": "UNIMAP_label_embedding",
    "num_layers":3,
    "dropout": 0.3,
    "learning_rate": 0.001,
    "weight_decay": 0.0005,
}

config = edict(config)



Dataset = namedtuple("Dataset", 
               ["graph", "num_classes", "train_index",
                "train_label", "valid_index", "valid_label", "test_index"])

def load_edges(num_nodes, self_loop=True, add_inverse_edge=True):
    # 从数据中读取边
    edges = pd.read_csv("data/data61620/edges.csv", header=None, names=["src", "dst"]).values

    if add_inverse_edge:
        edges = np.vstack([edges, edges[:, ::-1]])

    if self_loop:
        src = np.arange(0, num_nodes)
        dst = np.arange(0, num_nodes)
        self_loop = np.vstack([src, dst]).T
        edges = np.vstack([edges, self_loop])
    
    return edges

def load():
    # 从数据中读取点特征和边，以及数据划分
    node_feat = np.load("data/data61620/feat.npy")
    num_nodes = node_feat.shape[0]
    edges = load_edges(num_nodes=num_nodes, self_loop=True, add_inverse_edge=True)
    graph = pgl.graph.Graph(num_nodes=num_nodes, edges=edges, node_feat={"feat": node_feat})
    
    indegree = graph.indegree()
    norm = np.maximum(indegree.astype("float32"), 1)
    norm = np.power(norm, -0.5)
    graph.node_feat["norm"] = np.expand_dims(norm, -1)
    
    df = pd.read_csv("data/data61620/train.csv")
    node_idx = df["nid"].values
    node_label = df["label"].values
    train_part = int(len(node_idx) * 0.8)
    train_index = node_idx[:train_part]
    train_label = node_label[:train_part]
    valid_index = node_idx[train_part:]
    valid_label = node_label[train_part:]
    test_index = pd.read_csv("data/data61620/test.csv")["nid"].values
    dataset = Dataset(graph=graph, 
                    train_label=train_label,
                    train_index=train_index,
                    valid_index=valid_index,
                    valid_label=valid_label,
                    test_index=test_index, num_classes=35)
    return dataset

dataset = load()

test_index = dataset.test_index
test_index = np.expand_dims(test_index, -1)
test_label = np.zeros((len(test_index), 1), dtype="int64")

val_index = dataset.valid_index
val_label = np.reshape(dataset.valid_label, [-1, 1])
val_index = np.expand_dims(val_index, -1)

train_index = dataset.train_index
train_label = np.reshape(dataset.train_label, [-1 , 1])
train_index = np.expand_dims(train_index, -1)
print(train_label.shape)
print(train_index[:int(len(train_index)*0.625)].shape)


(56188, 1)
(35117, 1)


In [3]:

def linear(input, hidden_size, name, with_bias=True):
    """linear"""
    fan_in=input.shape[-1]
    bias_bound = 1.0 / math.sqrt(fan_in)
    if with_bias:
        fc_bias_attr = F.ParamAttr(initializer=F.initializer.UniformInitializer(low=-bias_bound, high=bias_bound))
    else:
        fc_bias_attr = False
        
    negative_slope = math.sqrt(5)
    gain = math.sqrt(2.0 / (1 + negative_slope ** 2))
    std = gain / math.sqrt(fan_in)
    weight_bound = math.sqrt(3.0) * std
    fc_w_attr = F.ParamAttr(initializer=F.initializer.UniformInitializer(low=-weight_bound, high=weight_bound))

    output = L.fc(input,
        hidden_size,
        param_attr=fc_w_attr,
        name=name,
        bias_attr=fc_bias_attr)
    return output


def transformer_gat_pgl(gw,
        feature,
        hidden_size,
        name,
        num_heads=4,
        attn_drop=0,
        edge_feature=None,
        concat=True,
        is_test=False):
    '''transformer_gat_pgl
    '''

    def send_attention(src_feat, dst_feat, edge_feat):
        if edge_feat is None or not edge_feat:
            output = src_feat["k_h"] * dst_feat["q_h"]
            output = L.reduce_sum(output, -1)
            output = output / (hidden_size ** 0.5)
            return {"alpha": output, "v": src_feat["v_h"]}   # batch x h     batch x h x feat
        else:
            edge_feat = edge_feat["edge"]
            edge_feat = L.reshape(edge_feat, [-1, num_heads, hidden_size])
            output = (src_feat["k_h"] + edge_feat) * dst_feat["q_h"]
            output = L.reduce_sum(output, -1)
            output = output / (hidden_size ** 0.5)
            return {"alpha": output, "v": (src_feat["v_h"] + edge_feat)}   # batch x h     batch x h x feat

    def reduce_attention(msg):
        alpha = msg["alpha"]  # lod-tensor (batch_size, seq_len, num_heads)
        h = msg["v"]
        alpha = paddle_helper.sequence_softmax(alpha)
        old_h = h
        
        if attn_drop > 1e-15:
            alpha = L.dropout(
                alpha,
                dropout_prob=attn_drop,
                is_test=is_test,
                dropout_implementation="upscale_in_train")
        h = h * alpha
        h = L.lod_reset(h, old_h)
        h = L.sequence_pool(h, "sum")
        if concat:
            h = L.reshape(h, [-1, num_heads * hidden_size])
        else:
            h = L.reduce_mean(h, dim=1)
        return h
    
    q_w_attr=F.ParamAttr(initializer=F.initializer.XavierInitializer())
    q_bias_attr=F.ParamAttr(initializer=F.initializer.ConstantInitializer(0.0))
    q = L.fc(feature,
            hidden_size * num_heads,
            name=name + '_q_weight',
            param_attr=q_w_attr,
            bias_attr=q_bias_attr)

    k_w_attr=F.ParamAttr(initializer=F.initializer.XavierInitializer())
    k_bias_attr=F.ParamAttr(initializer=F.initializer.ConstantInitializer(0.0))
    k = L.fc(feature,
            hidden_size * num_heads,
            name=name + '_k_weight',
            param_attr=k_w_attr,
            bias_attr=k_bias_attr)

    v_w_attr=F.ParamAttr(initializer=F.initializer.XavierInitializer())
    v_bias_attr=F.ParamAttr(initializer=F.initializer.ConstantInitializer(0.0))
    v = L.fc(feature,
            hidden_size * num_heads,
            name=name + '_v_weight',
            param_attr=v_w_attr,
            bias_attr=v_bias_attr)
    
    reshape_q = L.reshape(q, [-1, num_heads, hidden_size])
    reshape_k = L.reshape(k, [-1, num_heads, hidden_size])
    reshape_v = L.reshape(v, [-1, num_heads, hidden_size])

    msg = gw.send(
        send_attention,
        nfeat_list=[("q_h", reshape_q), 
                    ("k_h", reshape_k),
                    ("v_h", reshape_v)],
        efeat_list=edge_feature)
    output = gw.recv(msg, reduce_attention)

    return output




In [4]:

import math
import paddle.fluid.layers as L
import paddle.fluid as F
from pgl.utils import paddle_helper

class Model(object):
    """
    UNIMAP_LABEL_EMBEDDING
    """
    def __init__(self,dataset,phase):
        '''
        UNIMAP_label_embedding
        '''
        self.out_size=40
        self.num_class = dataset.num_classes
        self.num_layers = config.get("num_layers", 3)
        self.hidden_size = config.get("hidden_size",128)
        self.dropout = config.get("dropout", 0.3)
        self.num_heads = config.get("num_heads", 2)
        self.edge_dropout = config.get("edge_dropout", 0.0)
        self.embed_size = 100

        self.gw =pgl.graph_wrapper.GraphWrapper(name="graph",
                                                node_feat=dataset.graph.node_feat_info())
        self.node_index = F.data('node_index',
                                shape=[None,1],
                                dtype='int64',)
        self.node_label = F.data("node_label", 
                                shape=[None, 1],
                                dtype="int64", )
        self.feature = self.gw.node_feat['feat']
        self.phase = phase

    def build_model(self):
        graph_wrapper = self.gw
        node_label = self.node_label
        node_index = self.node_index
        phase = self.phase
        label_feature = self.label_embed_input(self.feature)
        if phase == "train": 
            edge_dropout = self.edge_dropout
        else:
            edge_dropout = 0

        feature_batch = L.dropout(label_feature, dropout_prob=self.dropout, 
                                dropout_implementation='upscale_in_train')
        for i in range(self.num_layers - 1):
            feature_batch=self.get_gat_layer(i, graph_wrapper, feature_batch, 
                                             hidden_size=self.hidden_size,
                                             num_heads=self.num_heads, 
                                             concat=True, 
                                             layer_norm=True, relu=True)#,gate=True)
            if self.dropout > 0:
                feature_batch = L.dropout(feature_batch, dropout_prob=self.dropout, 
                                     dropout_implementation='upscale_in_train') 
            
            
        feature_batch = self.get_gat_layer(self.num_layers - 1, graph_wrapper, feature_batch, 
                                           hidden_size=self.out_size, num_heads=self.num_heads, 
                                             concat=False, layer_norm=False, relu=False, gate=True)
        
        
        if phase=='train':
            unlabel_idx = F.data("unlabel_idx",shape=[None],dtype='int64')
            node_label = fluid.layers.gather(node_label,unlabel_idx)
            node_index = fluid.layers.gather(node_index,unlabel_idx)

        logits = feature_batch
        pred = fluid.layers.gather(logits, node_index)
        loss,pred = fluid.layers.softmax_with_cross_entropy(
            logits=pred, label=node_label, return_softmax=True)
        acc = fluid.layers.accuracy(input=pred, label=node_label, k=1)
        pred = fluid.layers.argmax(pred, -1)
        loss = fluid.layers.mean(loss)

        if phase == "train":
            adam = fluid.optimizer.Adam(
                learning_rate=config.learning_rate,
                regularization=fluid.regularizer.L2DecayRegularizer(
                    regularization_coeff=config.weight_decay))

            adam.minimize(loss)
        
        return loss,acc,pred

    def label_embed_input(self, feature):   
        label = F.data('train_label',shape=[None,1],dtype='int64')
        label_idx = F.data(name='label_idx', shape=[None], dtype="int64")
        label = L.reshape(label, shape=[-1])
        label = L.gather(label, label_idx, overwrite=False)

        node_idx = F.data('train_index',shape=[None,1],dtype='int64')
        node_idx = L.gather(node_idx,label_idx,overwrite=False)
        node_idx = L.reshape(node_idx,shape=[-1])

        embed_attr = F.ParamAttr(initializer=F.initializer.NormalInitializer(loc=0.0, scale=1.0))
        embed = F.embedding(input=label, size=(self.out_size, self.embed_size), param_attr=embed_attr )
        
        feature_label = L.gather(feature, node_idx, overwrite=False)
        feature_label = feature_label + embed
        feature = L.scatter(feature, node_idx, feature_label, overwrite=True)
        
        
        lay_norm_attr = F.ParamAttr(initializer=F.initializer.ConstantInitializer(value=1))
        lay_norm_bias = F.ParamAttr(initializer=F.initializer.ConstantInitializer(value=0))
        feature = L.layer_norm(feature, name='layer_norm_feature_input', 
                                      param_attr=lay_norm_attr, 
                                      bias_attr=lay_norm_bias)
        
        return feature
    def get_gat_layer(self, i, gw, feature, hidden_size, num_heads, concat=True,
                      layer_norm=True, relu=True, gate=False):
        
        fan_in = feature.shape[-1]
        bias_bound = 1.0 / math.sqrt(fan_in)
        fc_bias_attr = F.ParamAttr(initializer=F.initializer.UniformInitializer(low=-bias_bound, high=bias_bound))
        
        negative_slope = math.sqrt(5)
        gain = math.sqrt(2.0 / (1 + negative_slope ** 2))
        std = gain / math.sqrt(fan_in)
        weight_bound = math.sqrt(3.0) * std
        fc_w_attr=F.ParamAttr(initializer=F.initializer.UniformInitializer(low=-weight_bound, high=weight_bound))
        
        if concat:
            skip_feature = L.fc(feature,
                            hidden_size*num_heads,
                            param_attr=fc_w_attr,
                            name='fc_skip_' + str(i),
                            bias_attr=fc_bias_attr)
        else:
            skip_feature = L.fc(feature,
                            hidden_size,
                            param_attr=fc_w_attr,
                            name='fc_skip_' + str(i),
                            bias_attr=fc_bias_attr)
        out_feat = transformer_gat_pgl(gw, feature, hidden_size, 'gat_' + str(i), num_heads, concat=concat,) 

        
        if gate: 
          
            fan_in = out_feat.shape[-1] * 3
            bias_bound = 1.0 / math.sqrt(fan_in)
            fc_bias_attr = F.ParamAttr(initializer=F.initializer.UniformInitializer(low=-bias_bound, high=bias_bound))

            negative_slope = math.sqrt(5)
            gain = math.sqrt(2.0 / (1 + negative_slope ** 2))
            std = gain / math.sqrt(fan_in)
            weight_bound = math.sqrt(3.0) * std
            fc_w_attr = F.ParamAttr(initializer=F.initializer.UniformInitializer(low=-weight_bound, high=weight_bound))

            
            
            gate_f = L.fc([skip_feature, out_feat, out_feat - skip_feature],
                            1,
                            param_attr=fc_w_attr,
                            name='gate_' + str(i),
                            bias_attr=fc_bias_attr)
            
            gate_f = L.sigmoid(gate_f)
            
            out_feat = skip_feature * gate_f + out_feat * (1 - gate_f)

        else:
            out_feat = out_feat + skip_feature
            
            
        if layer_norm:
            lay_norm_attr = F.ParamAttr(initializer=F.initializer.ConstantInitializer(value=1))
            lay_norm_bias = F.ParamAttr(initializer=F.initializer.ConstantInitializer(value=0))
            out_feat = L.layer_norm(out_feat, name='layer_norm_' + str(i), 
                                      param_attr=lay_norm_attr, 
                                      bias_attr=lay_norm_bias)
        
                                     
        if relu:
            out_feat = L.relu(out_feat)
        return out_feat


In [5]:
import pgl
import paddle.fluid as fluid
import numpy as np
import time

use_cuda =True
place = fluid.CUDAPlace(0) if use_cuda else fluid.CPUPlace()

train_program = fluid.Program()
startup_program = fluid.Program()
test_program = fluid.Program()

with fluid.program_guard(train_program, startup_program):
    with fluid.unique_name.guard():
        model = Model(dataset,'train')
        loss,acc,pred= model.build_model()

with fluid.program_guard(test_program, startup_program):
    with fluid.unique_name.guard():
        model = Model(dataset,'test')
        v_loss, v_acc, v_pred = model.build_model()
        

test_program = test_program.clone(for_test=True)

exe = fluid.Executor(place)
exe.run(startup_program)

In [6]:
epochs = 3000

# 将图数据变成 feed_dict 用于传入Paddle Excecutor
label_rate = 0.625
best_val_acc = 0
index_idx = np.array([*range(len(train_index))])
feed_dict = model.gw.to_feed(dataset.graph)
feed_dict["train_label"] = np.array(train_label, dtype="int64")
feed_dict["train_index"] = np.array(train_index,dtype='int64')
for epoch in range(epochs):
    tmp_idx = index_idx
    np.random.shuffle(tmp_idx)
    label_idx=tmp_idx[ :int(label_rate*len(tmp_idx))]
    unlabel_idx= tmp_idx[int(label_rate*len(tmp_idx)): ]
    feed_dict['label_idx']= np.array(label_idx,dtype='int64')
    feed_dict["unlabel_idx"]= np.array(unlabel_idx,dtype='int64')
    feed_dict["node_label"] = np.array(train_label,dtype='int64')
    feed_dict['node_index'] = np.array(train_index,dtype='int64')

    
    train_loss,train_acc = exe.run(train_program,
                    feed=feed_dict,
                    fetch_list=[loss,acc],
                    return_numpy=True)
    feed_dict['label_idx'] = np.array(index_idx,dtype='int64')
    feed_dict["node_index"] = np.array(val_index, dtype="int64")
    feed_dict["node_label"] = np.array(val_label, dtype="int64")
    val_loss, val_acc = exe.run(test_program,
                            feed=feed_dict,
                            fetch_list=[v_loss, v_acc],
                            return_numpy=True)
    print("Epoch", epoch,"Train Acc", train_acc[0],"Valid Acc", val_acc[0])
    if (val_acc[0] > best_val_acc):
        best_val_acc = val_acc[0]
        fluid.save(train_program, "best/best")

In [ ]:
feed_dict["node_index"] = np.array(test_index, dtype="int64")
feed_dict["node_label"] = np.array(test_label, dtype="int64") #假标签
fluid.load(test_program,'best/best',exe)
test_prediction = exe.run(test_program,
                            feed=feed_dict,
                            fetch_list=[v_pred],
                            return_numpy=True)[0]
print("test Acc", test_prediction[0])

In [ ]:
"""submission = pd.DataFrame(data={
                            "nid": test_index.reshape(-1),
                            "label": test_prediction.reshape(-1)
                        })
submission.to_csv("submission_file/{}.csv".format(best_val_acc), index=False)
submission.to_csv('submission.csv',index=False)
print(best_val_acc)"""

In [ ]:
"""import csv
from collections import Counter

def vote_merge(filelst):
    result = {}
    fw = open('merge.csv', encoding='utf-8', mode='w', newline='')
    csv_writer = csv.writer(fw)
    csv_writer.writerow(['nid', 'label'])
    for filepath in filelst:
        cr = open(filepath, encoding='utf-8', mode='r')
        csv_reader = csv.reader(cr)
        for i, row in enumerate(csv_reader):
            if i == 0:
                continue
            idx, cls = row
            if idx not in result:
                result[idx] = []
            result[idx].append(cls)

    for nid, clss in result.items():
        counter = Counter(clss)
        true_cls = counter.most_common(1)
        csv_writer.writerow([nid, true_cls[0][0]])

if __name__ == '__main__':
    vote_merge([
        # r"E:\学习资料\PGL\论文节点\submission_0.715.csv",
        #         "E:\学习资料\PGL\论文节点\submission_0.6836.csv",
        #         "E:\学习资料\PGL\论文节点\submission_0.7153.csv",
        #         "E:\学习资料\PGL\论文节点\submission_0.68744.csv",
        #         "E:\学习资料\PGL\论文节点\submission_0.70872.csv",
        #         "E:\学习资料\PGL\论文节点\submission_0.71539.csv",
                "E:\学习资料\PGL\论文节点\submission_val_0.7479889.csv",
                "E:\学习资料\PGL\论文节点\submission_val_0.74706346.csv",
                "E:\学习资料\PGL\论文节点\submission_val_0.7459244.csv",
                "E:\学习资料\PGL\论文节点\submission_val_0.7433616.csv",
        #         "E:\学习资料\PGL\论文节点\submission_0.73469.csv",
        #         "E:\学习资料\PGL\论文节点\submission_0.725.csv",
        #         "E:\学习资料\PGL\论文节点\submission_0.721.csv",
        "E:\学习资料\PGL\论文节点\submission_resgat_val_0.755535.csv",
        "E:\学习资料\PGL\论文节点\submission_resgat_val_0.7546807.csv",
        "E:\学习资料\PGL\论文节点\submission_resgat_val_0.75290096.csv",
        "E:\学习资料\PGL\论文节点\submission_resgat_val_0.75581974.csv",
        "E:\学习资料\PGL\论文节点\submission_resgat_val_0.7537553.csv",
        "E:\学习资料\PGL\论文节点\submission_resgat_val_0.7526162.csv",
        "E:\学习资料\PGL\论文节点\submission_resgat_val_0.7527.csv",
        "E:\学习资料\PGL\论文节点\submission_resgatii_val_0.75033814.csv",
        "E:\学习资料\PGL\论文节点\submission_resgatii_val_0.75090766.csv",
        "E:\学习资料\PGL\论文节点\submission_resgat_val_0.75589097.csv",
        "E:\学习资料\PGL\论文节点\submission_resgat_val_0.7536129.csv",
        "E:\学习资料\PGL\论文节点\submission_resgat_val_0.754.csv",
        "E:\学习资料\PGL\论文节点\submission_resgat_val_0.7548231.csv",
        "E:\学习资料\PGL\论文节点\submission_resgat_val_0.75596213.csv",
        "E:\学习资料\PGL\论文节点\submission_resgat_val_0.7563181.csv",
                ])"""